In [1]:
import json
from tqdm import tqdm
with open('data/raw_data.json') as f:
        data = json.load(f)
print(len(data))

705


In [2]:
# Number of attributes
attributes = []

for sample in data:
    disease = list(sample.keys())[0]      
    for att_obj in sample[disease]:
        for k , v in att_obj.items():
            if k not in attributes:
                attributes.append(k)
print(attributes)
print(len(attributes)) # 8

['disease-description', 'disease-causes', 'disease-symptoms_free', 'disease-overview', 'disease-prevention', 'disease-diagnosis', 'disease-treatment', 'disease-treatment_summary']
8


## Turn raw data to structured data
- Sentences
- Rename 'disease-overview' -> 'disease-risk-factor'
- Get short answer for each attribute
- Unique key
- Synonyms

In [3]:
# Format data to list of disease

DATA = []
for sample in data:
    for disease, _ in sample.items():
        
        disease_object = {}
        disease_object['disease'] = disease
        for att_obj in sample[disease]:
            for k,v in att_obj.items():
                disease_object[k] = v
        DATA.append(disease_object)

In [4]:
# split into sentences and rename

for disease_object in DATA:
    for k,v in disease_object.items():
        disease_object[k] = v.split('\n')

In [5]:
DATA[123]

{'disease': ['gioi leo'],
 'disease-description': ['Giời leo là một trong những bệnh phổ biến hiện nay, hầu hết mọi người ai cũng đã từng mắc phải. Vậy giời leo là bệnh gì?',
  'Giời leo (hay còn gọi là bệnh zona) là một bệnh thường kèm theo những đau đớn kéo dài từ 6 tháng đến vài năm. Để hạn chế nguy cơ mắc bệnh, hiện nay đã có vắc xin tiêm phòng có thể phòng được cả bệnh thủy đậu và bệnh giời leo.',
  'Bệnh giời leo thường xuất hiện vào mùa mưa với thời tiết lạnh, độ ẩm cao kết hợp với cơ địa mệt mỏi, sức đề kháng yếu.'],
 'disease-causes': ['Giời leo là bệnh viêm dây thần kinh do virus nhóm Herpes gây ra, có thể xuất hiện ở bất cứ đâu trên cơ thể con người nhưng thường gặp nhất là ở vùng liên sườn, gần tai và đùi trong. Ngoài ra, bệnh còn xuất hiện ở bụng, cổ, vai, mặt, lưng, nguy hiểm nhất và khó điều trị nhất là ở hố mắt.'],
 'disease-symptoms_free': ['Bệnh giời leo có những dấu hiệu và triệu chứng bệnh như sau:',
  'Tổn thương da đau rát như bị trầy xước hay bỏng, ngứa râm ran g

In [6]:
# get short answer and filter database
import re

def get_short_answer(li_of_sentence, att=None):
    ''' Get short answer for definition base on att
    Args:
        - li_of_sentence (list) of sentences
        - att (str) : attribute of given disease
    Return:
    
    '''
    result = li_of_sentence
    if att == None:
        return result

    
    if att == 'disease-description':
        # the near-by sentence of question
        for i, sent in enumerate(li_of_sentence):
            # is question
            if '?' in sent and i == 0:
                pattern = ".+?(?= là)"
                disease = re.search(pattern, sent)
                if disease == None:
                    try:
                        result = li_of_sentence[i+1]
                    except:
                        print(i)
                        print(li_of_sentence)
                    return result 
                
                disease = disease.group(0).lower()
                result = li_of_sentence[i+1:]
                # get answer sentence
                for sent in result:
                    sent_ = sent.lower()
                    if sent_.find('là')!= -1 and sent_.find(disease) != -1 and sent_.find(disease) < sent_.find('là'):
                        result = [sent]
                        break
                break
                
    if att in ['disease-overview', 'disease-prevention','disease-causes', 'disease-symptoms_free','disease-diagnosis','disease-treatment','disease-treatment_summary']:
        if '?' in li_of_sentence[0]:
            result = li_of_sentence[1:]
        else:
            for i, sent in enumerate(li_of_sentence):
                # is question
                if ':' in sent:
                    result = li_of_sentence[i:]
                    break
                
    return result

def filter_noise(li_of_sentence):
    ''' Remove questions/noise sentences in given list
    Args:
        - li_of_sentence (list):
    '''
        # remove "read more"
    for i, sent in enumerate(li_of_sentence):
        # is question
        if 'xem thêm' in sent.lower() or 'xem thêm:' in sent.lower():
            li_of_sentence = li_of_sentence[:i]
            break
    return li_of_sentence

In [8]:
FILTERED_DATA = []

for disease_object in DATA:
    d_object = {}
    for k,v in disease_object.items():
        d_object[k] = v
        if k != 'disease':
            d_object[k] = filter_noise(v)
            d_object[k] = get_short_answer(v,k)
           
        
#     # symnonym
#     d_object['synonym'] = get_synonym(d_object['disease'][0],d_object['disease-description'])
#     # unique key
#     d_object['key'] = get_unique_key(d_object)
            
    FILTERED_DATA.append(d_object)

In [26]:
print(FILTERED_DATA[32]['disease-description'])

['Bệnh Basedow xảy ra ở phụ nữ nhiều hơn, chiếm tới 80% các trường hợp, thường ở độ tuổi từ 20-50 và bệnh nhân có tiền sử gia đình biểu hiện bệnh tuyến giáp.', 'Bệnh basedow rất nguy hiểm cho hệ tim mạch, nếu tình trạng bệnh kéo dài mà không được điều trị đúng cách thì bệnh dễ dẫn tới biến chứng bão giáp khiến bệnh nhân tử vong trong tình trạng suy tim.']


In [219]:
print(FILTERED_DATA[0]['disease-description'])
print('==========================')
# print(FILTERED_DATA[0]['disease-description-short'])

['Rối loạn tiền đình là tình trạng tổn thương dây thần kinh số 8 do nhiều nguyên nhân khác nhau khiến thông tin dẫn truyền bị sai lệch làm cho cơ thể mất khả năng kiểm soát thăng bằng, hoa mắt, chóng mặt, ù tai, buồn nôn…']


In [220]:
with open('data.json', 'w') as f:
    json.dump(FILTERED_DATA, f, ensure_ascii=False)

### Get disease,question,relation

In [223]:
import pandas as pd

def get_question(li_of_sentences):
    ''' Return list of questions
    '''
    result = []
    for sent in li_of_sentences:
        if '?' in sent:
            result.append(sent)
    return result

In [226]:
'''
[
    {
        'disease' : (str),
        'questions' : [
            {
                'question' : (list of str) 
                'relation' : (str)
                'context' :  (str)
            }
        ]
    }
    
]
'''
QUESTION_DATA = []

for disease_object in DATA:
    d_object = {}
    d_object['disease'] = disease_object['disease']
    d_object['questions'] = []
    for k,v in disease_object.items():
        if k != 'disease':
            question = get_question(v)
            if question != []:
                d_object['questions'].append({
                    'question' : question,
                    'relation' : k,
                    'context' : v
                })
    if d_object['questions'] != []:
        QUESTION_DATA.append(d_object)

In [230]:
with open('question_data.json', 'w') as f:
    json.dump(QUESTION_DATA, f, ensure_ascii=False)

## Entity Linking
1. english
2. main type
3. related type

In [40]:
def get_synonym(disease, disease_description):
    ''' Return list of synonym
    Args:
        - disease (str) : non-acent
        - disease-description (list) of sentences
    '''
    # get vietnamese disease
    vi_disease = get_vi_disease(disease,disease_description)
    
    # get synonyms : E
    # TODO
    
    return vi_disease

def find_synonyms(disease):
    '''
    - main type
    - related type
    - synonym 1-1
    - english
    Args:
        - disease (str)
    '''
def remove_accent(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

def get_vi_disease(disease,disease_description):
    disease = disease.replace('benh ','').strip()
    # find disease in non-accent description
    for sent in disease_description:
        sent_ = remove_accent(sent.lower())
        start_idx = sent_.find(disease)
        if start_idx != -1:
            return sent[start_idx : start_idx + len(disease)].lower()
    return disease

In [41]:
# Unique key
import hashlib

def get_unique_key(value):
    ''' Return a hash for given dict
    Args:
        - value (dict)
    '''
    return hashlib.sha1(json.dumps(value, sort_keys=True).encode()).hexdigest()

a = FILTERED_DATA[0]

print(get_unique_key(a))

b = FILTERED_DATA[1]

print(get_unique_key(b))

b3c502daa3a87688982f331e609d13df0f7243b6
3606ce090e6a5002ab0d14985c1ef6c166b896e1


In [42]:
DISEASE = []
for sample in FILTERED_DATA:
    disease = get_vi_disease(sample['disease'][0],sample['disease-description'])
    DISEASE.append(disease)
print(len(DISEASE))

705


In [43]:
DISEASE[:5]

['rối loạn tiền đình',
 'lao phoi',
 'te bi tay chan',
 'viêm đại tràng',
 'giời leo']

In [44]:
FILTERED_DATA[1]['disease-description']

['Bệnh lao (còn gọi là TB) là một bệnh truyền nhiễm do vi trùng lao gây nên. Nếu vi trùng lao thâm nhập vào một cơ quan nào đó trong cơ thể và sinh sôi đồng thời cơ thể không thể chống lại nó, khi đó sẽ hình thành bệnh lao.']

In [2]:
from googletrans import Translator
translator = Translator()

In [3]:
translation = translator.translate('rối loạn tiền đình', dest='en')
print(translation.text)

vestibular disorders


In [1]:
!pip install mtranslate

/home/minh/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/minh/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3698 sha256=9024245361ce6067d014a622b3e4d39421ceb75291eaff7d54a1083cc50f1aa6
  Stored in directory: /home/minh/.cache/pip/wheels/99/49/c3/b1e6cb0d8b908238238a086ad2d0f30d4a7c692957e6cea74b
Successfully built mtranslate


In [3]:

from mtranslate import translate


to_translate = 'sinusitis maxillary'
print(translate(to_translate,'vi'))

viêm xoang hàm trên
